# MC Code RAG Workshop


Vector base and RAG test.

## Setup


### Hyper parameters


In [ ]:
# p1=0  #Parameter 1 


### Dependencies


In [ ]:
# %pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4 python-dotenv

### Key from .env


In [1]:
import os
from dotenv import load_dotenv,find_dotenv

# Use this line of code if you have a local .env file
load_dotenv(find_dotenv()) 

# Or set it like this
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Print this line to double check your API key
# print(os.environ["OPENAI_API_KEY"])

True

### LangSmith(enabled by default)


In [2]:
# Store all the keys in .env
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_KEY"] #getpass.getpass()
print(os.environ["LANGCHAIN_API_KEY"])

ls__7cb2a18f629e4c10b5a34d49223fba88


### Import from


In [3]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyPDFLoader, PyPDFium2Loader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document

### Load File


In [ ]:
# Web loader
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

In [ ]:
# Pdf loader
file_path = './docs/WMX3SampleCodes.pdf'
# file_path = 'nais2023.pdf'
# loader = PyPDFLoader(file_path)
# loader = PyPDFLoader(file_path, extract_images=True)   #extract images as text as well

loader = PyPDFium2Loader(file_path,  extract_images=False) 
# docs = loader.load_and_split()
docs = loader.load()
print(docs[0])


In [ ]:
len(docs)


In [4]:
# Txt loader
loader = TextLoader("./docs/WMX3UserManual_a.txt")
docs = loader.load()
# docs[0].page_content[:100000]
len(docs)

1

In [3]:
# Txt loader
loader = TextLoader("./docs/WMX3SampleCodes.py")
docs = loader.load()
# docs[0].page_content[:100000]
len(docs)

1

In [3]:
# Txt loader for api with instruction
loader = TextLoader("./docs/WMX3API_CleanedData.json")
docs = loader.load()
# docs[0].page_content[:100000]
len(docs)

1

### Chunk


In [5]:
#WMX3manual text chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [4]:
#Sample code chunk with `` separators
separators = ['``']  # Adjust based on actual document structure
text_splitter = RecursiveCharacterTextSplitter(separators=separators, keep_separator=True, chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [7]:
#Sample code chunk with } separators -api with instruction, WMX3API_CleanedData.json
separators = ['}']  # Adjust based on actual document structure
text_splitter = RecursiveCharacterTextSplitter(separators=separators, keep_separator=True, chunk_size=1000, chunk_overlap=200, add_start_index=True)
splits = text_splitter.split_documents(docs)

In [6]:
splits[0]

Document(page_content='Main \n\n\nThis help file contains information regarding the functions that are available in WMX3. \nUse the links to the left to navigate through this help file. \n \n\nSupport Specifications \n\n\nWMX3 RTX Version \n\n \n\n\nWMX3 RTX Version \n\n\nSupported PC Hardware \nSupported OS \nSupported NIC \nSupported Library and IDE \nSupported Sample Project \nSupport for user application development running only on a real-time OS \nSupport for user application development running only on a non real-time \nOS \n \n\n\nSupported PC Hardware \n\nTo operate WMX3, a PC with the following specifications is required. \n\nCPU \nRequirement', metadata={'source': './docs/WMX3UserManual_a.txt', 'start_index': 0})

In [7]:
splits[1]

Document(page_content='Supported PC Hardware \n\nTo operate WMX3, a PC with the following specifications is required. \n\nCPU \nRequirement \n\n\nPerformance : A CPU with performance of at least the Atom E3845 (2GHz) is \nrequired. \nNumber of cores : Since it is necessary to assign a dedicated CPU core to RTX, a \nminimum of 2 cores (excluding Hyper-Threading virtual cores) is required. \nGeneration : Each version of RTX supports specific CPU generations. Select the \nCPU based on the compatibility information below. CPUs that are not listed in the \ncompatibility information are not supported at that time. \nFor additional information, please refer to the following technical note of IntervalZero \n(the developer of RTX). \n\nhttp://softservo.com/pub/RTX_Doc/RTX64/Manual/RTX64_Processor_Compatibility.pdf \n\nRecommended', metadata={'source': './docs/WMX3UserManual_a.txt', 'start_index': 492})

In [ ]:
help(RecursiveCharacterTextSplitter)

### Vectorstore - chroma


In [8]:
embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   #text-embedding-3-large   #text-embedding-ada-002    #text-embedding-3-small

# If txt vectorstore exists
# if os.path.exists("Vectorstore/chromadb-MCCoder"):
        # vectorstore = Chroma(
                #     embedding_function=embedding_model,
                #     persist_directory="Vectorstore/chromadb",
                #     ) 
# else:
        # Load from chunks and save to disk
        # vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory="Vectorstore/chromadb") 

# If vectorstore exists
vectorstore_path = "Vectorstore/chromadb-MCCoder"
 # Load from chunks and save to disk
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model,  persist_directory=vectorstore_path) 
# if os.path.exists(vectorstore_path):
#         vectorstore = Chroma(
#                     embedding_function=embedding_model,
#                     persist_directory=vectorstore_path,
#                     ) 
# else:
#         # Load from chunks and save to disk
#         vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model, persist_directory=vectorstore_path) 


In [9]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
# retriever = vectorstore.similarity_search_with_score('a typical python code of WMX3 for a axis/servo/motor to move or do positioning.')

In [10]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x10fa47ad0>, search_kwargs={'k': 4})

In [11]:
# retrieved_docs = retriever.invoke("a typical python code of WMX3 for a axis/servo/motor to move or do positioning. ")

retrieved_docs = retriever.invoke("Supported PC Hardware ")

#print(retrieved_docs[0].page_content)
retrieved_docs 

[Document(page_content='Supported PC Hardware \n\nTo operate WMX3, a PC with the following specifications is required. \n\nCPU \nRequirement \n\n\nPerformance : A CPU with performance of at least the Atom E3845 (2GHz) is \nrequired. \nNumber of cores : Since it is necessary to assign a dedicated CPU core to RTX, a \nminimum of 2 cores (excluding Hyper-Threading virtual cores) is required. \nGeneration : Each version of RTX supports specific CPU generations. Select the \nCPU based on the compatibility information below. CPUs that are not listed in the \ncompatibility information are not supported at that time. \nFor additional information, please refer to the following technical note of IntervalZero \n(the developer of RTX). \n\nhttp://softservo.com/pub/RTX_Doc/RTX64/Manual/RTX64_Processor_Compatibility.pdf \n\nRecommended', metadata={'source': './docs/WMX3UserManual_a.txt', 'start_index': 492}),
 Document(page_content='Generation : Intel 2nd gen or later, Corei3, Corei5, Corei7 are rec

### Vectorstore - pinecone


In [ ]:
from langchain_pinecone import PineconeVectorStore
index_name = os.environ["PINECONE_INDEX_NAME"]


In [ ]:
# PineconeVectorStore.delete(delete_all=True)

In [ ]:

docsearch = PineconeVectorStore.from_documents(splits, embedding_model, index_name=index_name)


In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding_model)


In [ ]:
query = "what is the Architecture of wmx3?"
retrieved_docs = docsearch.similarity_search(query,k=8)
retrieved_docs

### Vectorstore - FAISS

In [ ]:
embedding_model=OpenAIEmbeddings(model="text-embedding-3-large")   

from langchain_community.vectorstores import FAISS
vectorstore_path = "Vectorstore/FAISS-pdf-images"
db = FAISS.from_documents(splits, embedding_model) 

In [ ]:
db.save_local(folder_path="Vectorstore/FAISS-pdf-images", index_name="myFaissIndex")

In [ ]:
db = FAISS.load_local(folder_path="Vectorstore/FAISS-pdf-images",embeddings=embedding_model,index_name="myFaissIndex")

In [ ]:
query = "a typical python code of WMX3 for a axis/servo/motor to move or do positioning."
docs = db.similarity_search(query)
docs

### Vectorstore - pathway

### serve Maximum marginal relevance search (MMR) Similarity search by vector ...

In [ ]:
from langchain_community.vectorstores import Qdrant
qdrant = Qdrant.from_documents(
    splits,
    embedding_model,
    path="Vectorstore/Qdrant-pdf-images",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

### Prompt template


In [ ]:
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Helpful Answer:"""


custom_rag_prompt = PromptTemplate.from_template(template)


In [ ]:
custom_rag_prompt = hub.pull("rlm/rag-prompt")

### Rag chain


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.1)   #gpt-4  #gpt-3.5-turbo

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)


In [ ]:
format_docs


In [ ]:

rag_chain.invoke("write an example code to close wmx3?")

In [ ]:
for chunk in rag_chain.stream("write a sample code to initialize wmx3?"): 
    print(chunk, end="", flush=True)

### Test RAG func


In [26]:
import re

def extract_code_sections(user_question):
    """
    Extracts numbered sections of a user question based on specific starting phrases.
    
    If the question starts with 'Write a python code', 'Python code', or 'write python' (case insensitive),
    it splits the question into paragraphs that start with numbers (e.g., 1., 2., 3.) and adds 
    'Write python code to ' before each paragraph, removing the numbers. If the question does not start 
    with the specified phrases or does not contain numbered lists, the entire question is saved into a single 
    element array. If the question does not start with the specified phrases, NoCoder is set to 1.
    
    Args:
        user_question (str): The user's question.
    
    Returns:
        tuple: NoCoder (int), an array of strings with each element containing a code instruction or the entire question.
    """
    result = []
    NoCoder = 0
    # Check if the input starts with the specified prefixes
    if re.match(r'(?i)^(Write a python code|Python code|write python)', user_question):
        # Find all numbered paragraphs
        paragraphs = re.findall(r'(\d+\.\s*)(.*)', user_question)
        if paragraphs:
            # Add 'Write python code to ' and remove numbers
            for _, para in paragraphs:
                result.append(f'Write python code to {para.strip()}')
        else:
            # Save the entire question to the array
            result.append(user_question)
    else:
        # Save the entire question to the array and set NoCoder to 1
        result.append(user_question)
        NoCoder = 1
    
    return NoCoder, result

# Example usage
user_question1 = """write python code to:
1. Read a file.
2. Process the data.
3. Save the result."""
user_question2 = "This is a general question about Python."
user_question3 = "Write a python code to read data without numbers."

print(extract_code_sections(user_question1))
print(extract_code_sections(user_question2))
print(extract_code_sections(user_question3))


(0, ['Write python code to Read a file.', 'Write python code to Process the data.', 'Write python code to Save the result.'])
(1, ['This is a general question about Python.'])
(0, ['Write a python code to read data without numbers.'])


In [ ]:
import openai

# Initialize the OpenAI client with your API key
client = openai.OpenAI(api_key='')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  temperature=0,
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message['content'].strip())


In [2]:
from openai import OpenAI
# client = openai.OpenAI(api_key='' )
client = OpenAI()
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a concept profound,\nRecursion dances, with elegance bound.\nA function calls itself, a loop of delight,\nUnraveling problems, layer by layer, taking flight.\n\nLike a mirror reflecting an image so true,\nRecursion looks back, revealing a clue.\nDividing a task into smaller bits,\nIt tackles complexity, piece by tiny bits.\n\nThrough branches and loops, it ventures on,\nA journey of beauty, until it is done.\nWith elegance and grace, it weaves its spell,\nUnraveling mysteries, the tale it tells.\n\nSo embrace the recursive, a friend so dear,\nIn the world of programming, have no fear.\nFor with each call, a problem unties,\nRecursion whispers, "In elegance lies."', role='assistant', function_call=None, tool_calls=None)
